## Global parameters

In [ ]:
shell_log=True
hostAddress="141-76-50-41.nip.io"

from tqdm import tqdm, tqdm_notebook
!pip install pymongo

### Mongo db & collection initialization

In [ ]:
from pymongo import MongoClient
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns

client = MongoClient("manager-mongo-manager")

db = client["fabric"]
collection_experiment = db["experiments"]


### Low level scripting functions

In [ ]:
import subprocess
import shlex
import datetime
from dateutil.tz import tzlocal
import pandas as pd
import glob
import time
from time import sleep 
def run_command_async(command):
    if (isinstance(command, list) == False):
        commands = [command]
    else:
        commands=command
    processes = [None] * len(commands)
    for i in range(len(commands)):
        processes[i] = subprocess.Popen(shlex.split(commands[i]), stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    return processes

def wait_for_command_async(processes):
    while processes:
        for i, process in enumerate(processes):
            output = process.stdout.readline().decode()
            if output == '' and process.poll() is not None:
                processes.remove(process)
                break
            if output:
                now=datetime.datetime.now(tzlocal())
                strnow = now.strftime("%Y-%m-%d %H:%M:%S")
                print ("Log {0} - {1} : ".format(i,strnow) + output.strip())
    
    rc = process.poll()
    return rc    
def run_command(command, shell=False, log=True):
    print(command)
    if (isinstance(command, list) == False):
        commands = [command]
    else:
        commands=command
    processes = [None] * len(commands)
    for i in range(len(commands)):
        if log == True:
            print (commands[i])
        processes[i] = subprocess.Popen(shlex.split(commands[i]), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=shell)
    while processes:
        for i, process in enumerate(processes):
            output = process.stdout.readline().decode()
            if output == '' and process.poll() is not None:
                processes.remove(process)
                break
            if output and log:
                now=datetime.datetime.now(tzlocal())
                strnow = now.strftime("%Y-%m-%d %H:%M:%S")
                print ("Log {0} - {1} : ".format(i,strnow) + output.strip())
    
    rc = process.poll()
    return rc
import os

class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)
def get_strnow():
    now=datetime.datetime.now(tzlocal())
    return now.strftime("%Y%m%d%H%M%S")


### Module scope functions

In [ ]:
def wait_for_http_status(url, code, duration):
    import requests
    import time
    print
    req_code = None
    start_time = time.time()
    end_time= time.time()
    while req_code != code and (end_time - start_time) < duration :
        try:
            time.sleep(1)
            r = requests.head(url)
            req_code = r.status_code
            print("\rWaiting status {} for {} seconds            ".format(code, end_time - start_time), end="\r", flush=True)
            return 0
        except requests.ConnectionError:
            print("failed to connect")
            return 1
        end_time = time.time()    

def wait_for_job(name, duration):
    with cd(".."):
        command = "./wait_for_job.sh {} {}".format(name, duration)
        result = run_command(command, log = shell_log)
        return result
        
def install_chart(path, name, params, timeout=120):
    
    run_command("helm init", log = shell_log)
       
    with cd(path):
        
        run_command("kubectl apply -f ..", log = shell_log)
        param_str = ""
        for k in params.keys():
            param_str += "--set {}={} ".format(str(k).replace("!","."), str(params[k]).replace("!","."))
        command = "helm install --name {} --namespace default . {} --wait --timeout {} ".format(name, param_str, timeout)
        result = run_command(command, log = shell_log)
        return result    
    
def uninstall_chart(name, timeout=120):
    with cd(sut_chart_directory):
        result = run_command("helm delete --purge {}".format(name), log = shell_log)
        if result == 0:
            sleep(timeout)
    return result

def launch_workflow(wf, scenario, additional_parameters={}):
    param_str = ""
    for k in additional_parameters.keys():
        param_str += "-p {}={} ".format(str(k).replace("!","."), str(additional_parameters[k]).replace("!","."))
    print(param_str)
    run_command("./argo submit --wait {} -n default -f {} {} ".format(wf, scenario, param_str))    
    
def clean_chart():
    run_command("helm delete moodle")    
    run_command("helm delete moodle-fdw")

### Test scope functions

In [ ]:

def deploy_test(params):
    # uninstall previous launches
    print(uninstall_stub())
    print(uninstall_injector())
    print(uninstall_recsys())

    # init naming for experiment
    strnow = get_strnow()
    params_recsys = params["params_recsys"]
    params_injector = params["params_injector"]
    
    params["name"] = params["name"] + strnow
    params_injector["name"] = params_injector["name"] + strnow
    params_recsys["name"] = params_recsys["name"] + strnow
    
    # add labels with names
    params_injector["experiment"] = params["name"] 
    params_recsys["experiment"] = params["name"] 
    params_recsys["proxy-sgx!experiment"] = params["name"]
    params_recsys["harness!experiment"] = params["name"]
    
    mongo_id = collection_experiment.insert_one(params).inserted_id
    collection_experiment.update_one({'_id':mongo_id}, {"$set": params}, upsert=False)

    # init step recsys
    params_recsys["start_time"] = time.time()
    params["status"] = "recsys"
    collection_experiment.update_one({'_id':mongo_id}, {"$set": params}, upsert=False)
    
    cpt = 0
    err = None
    while (err is None or err != 0) and cpt < 5:
        
        if params["stub"] != "1":
            err=install_recsys(params_recsys)
        else:
            err=install_stub(params_recsys)
        print("err={}".format(err))
        if err != 0:
            if params["stub"] != "1":
                uninstall_recsys()
            else:
                uninstall_stub()
        cpt += 1

    params_recsys["end_time"] = time.time()
    params["status"] = "recsys"
    if err != 0:
        params["status"] = "recsys_error"

    collection_experiment.update_one({'_id':mongo_id}, {"$set": params}, upsert=False)
    return params["status"]    
def run_test(params):
    # uninstall previous launches
    print(uninstall_injector())

    # init naming for experiment
    strnow = get_strnow()
    params_recsys = params["params_recsys"]
    params_injector = params["params_injector"]
    
    params["name"] = params["name"] + strnow
    params_injector["name"] = params_injector["name"] + strnow
    params_recsys["name"] = params_recsys["name"] + strnow
    
    # add labels with names
    params_injector["experiment"] = params["name"] 
    params_recsys["experiment"] = params["name"] 
    params_recsys["proxy-sgx!experiment"] = params["name"]
    params_recsys["harness!experiment"] = params["name"]
    
    mongo_id = collection_experiment.insert_one(params).inserted_id
    collection_experiment.update_one({'_id':mongo_id}, {"$set": params}, upsert=False)
   
    params_injector["start_time"] = time.time()
    params["status"] = "injector"
    collection_experiment.update_one({'_id':mongo_id}, {"$set": params}, upsert=False)
    if params["stub"] != "1":
        time.sleep(120)
    cpt = 0

    err = launch_injector(params_injector)
    print("Injector deployment err : {}".format(err))
    #uninstall_injector()

    if err != 0:
        if err == 1:
            # failed injector, we assume it is harness
            params["status"] = "harness_error"
        return params["status"]
    
    params_injector["end_time"] = time.time()
    params["status"] = "injector"
    collection_experiment.update_one({'_id':mongo_id}, {"$set": params}, upsert=False)

    params_injector["error"] = err
    params_injector["end_time"] = time.time()
    #now=datetime.datetime.now(tzlocal())
    #strnow = now.strftime("%Y%m%d%H%M%S")



    coll_name = "private-recsys.{}.locust_request_result".format(params_injector["name"]) 
    coll = db[coll_name]
    params["status"] = "finished"
    
    #if params["stub"] != "1":
    #    if is_harness_result_ok(coll) == False:
    #        params["status"] = "harness_error"
    #print("{} - {}".format(coll_name, is_harness_result_ok(coll)))
        
    params["end_time"] = time.time()
    collection_experiment.update_one({'_id':mongo_id}, {"$set": params}, upsert=False)
    return params["status"]


### Test functions

In [ ]:
import itertools
import collections
def unpack_dict(target):
    ar = collections.OrderedDict()
    for k in target.keys():
        if isinstance(target[k], list):
            ar[k] = target[k]
    print(ar.keys())
    combinations = [dict(zip(ar.keys(), i)) for i in itertools.product(*ar.values())]
    result = []
    for c in combinations:
        print(c)
        d = target.copy()
        d.update(c)
        result.append(d)
    return result, ar

def rename_db(old, new):
    import pymongo
    from pymongo import MongoClient
    client = MongoClient("manager-mongo-manager")
    client.admin.command('copydb',
                         fromdb=old,
                         todb=new)
    client.drop_database(old)
    
def loop_run_test(params_packed):
    list_params_recsys, arrays_params_recsys = unpack_dict(params_packed["params_recsys"])
    list_params_injector, arrays_params_injector = unpack_dict(params_packed["params_injector"])

    strnow = get_strnow()
    print("db:" + "-".join(list(arrays_params_recsys))+"_"+"-".join(list(arrays_params_injector))+"_"+strnow)

    print(list_params_injector)
    first=True
    for params_recsys in list_params_recsys:
        params = params_packed.copy()
        params["params_recsys"] = params_recsys.copy()
        result = deploy_test(params.copy())
        
        if result != "recsys":
            break
        for params_injector in list_params_injector:
            for i in range(5):
                params = params_packed.copy()
                params["params_recsys"] = params_recsys.copy()
                params["params_injector"] = params_injector.copy()
                if first == True:
                    print("First launch")
                    
                    result = run_test(params.copy())
                    if result != "finished":
                        print("Launch {} in error, redeploying recsys.".format(i))
                        result = deploy_test(params.copy())
                        
                    else:
                        first=False
                        break

                else:
                    print("Not first, no init/train needed")
                    params["params_injector"]["harness"] = False

                    result = run_test(params.copy())
                    if result != "harness_error":
                        break
    #rename database 
    #rename_db("private-recsys", "-".join(list(arrays_params_recsys))+"_"+"-".join(list(arrays_params_injector))+"_"+strnow)

def fusion_experiments_db(name_from, name_to, drop = False):
    import pymongo
    from pymongo import MongoClient
    client = MongoClient("manager-mongo-manager")    
    if drop == True:
        client.drop_database(name_to)
    from pymongo import MongoClient
    client = MongoClient("manager-mongo-manager")
    database_names = client.list_database_names()
    for database_name in database_names:
        if name_from in database_name:
            print(database_name)
            elements = list(client[database_name]["experiments"].find())
            client[name_to]["experiments"].insert_many(elements) 
            for coll_name in client[database_name].list_collection_names(nameOnly=True):
                if coll_name != "experiments":
                    elements = list(client[database_name][coll_name].find())
                    client[name_to][coll_name].insert_many(elements)
    
def remove_working_db():
    from pymongo import MongoClient
    client = MongoClient("manager-mongo-manager")   
    client.drop_database("fabric")
